In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
pd.set_option('display.max_columns',None)

In [21]:
train_df=pd.read_csv('Train_after_forward.csv',index_col='application_key')
X=train_df.iloc[:1000,:-1]
Y=train_df.iloc[:1000,-1]
train_df.head(3)

,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar13,mvar15,mvar16,mvar17,mvar18,mvar19,mvar20,mvar21,mvar22,mvar23,mvar25,mvar26,mvar27,mvar28,mvar29,mvar30,mvar31,mvar32,mvar33,mvar34,mvar35,mvar36,mvar37,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,default_ind
application_key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
230032,1696.0,1.6541,0.0,0.0,0.0,0.000000,6015.0,322.00000,40369.000000,18414.0,3067.000000,294080.148958,1.000000,1.000000,1.000000,3.0,3.0,94.780000,8987.180000,4305.929828,1462.000000,4532.000000,2890.000000,61.0,4532.0,1095.000000,3376.000000,625.000000,1.1667,1.0,0.000000,8.0,10.000000,4.0,1.0,73.780000,82.547000,0.08696,10.0,0.63899,0.070736,0.000000,0.0
230033,1846.0,0.8095,0.0,0.0,0.0,102.000000,7532.0,3171.00000,18234.000000,13664.0,1931.000000,294080.148958,0.000000,0.000000,0.000000,0.0,0.0,74.250000,953.060000,953.060000,1028.000000,2099.000000,2099.000000,30386.0,2281.0,2611.880274,2251.000000,169.000000,0.4167,0.0,0.000000,8.0,0.000000,2.0,0.0,99.129000,72.096758,0.00000,13.0,0.63836,0.070736,0.096135,1.0
230034,1745.0,0.4001,0.0,0.0,0.0,1606.702372,2536.0,6773.68666,34097.436013,2536.0,15429.679865,294080.148958,0.231582,0.266281,0.272026,0.0,0.0,57.790097,4423.856509,4305.929828,1750.065303,3369.121848,3096.580153,669.0,4623.0,3772.000000,3626.892986,470.430879,25.0833,0.0,0.198123,1.0,4.251362,1.0,0.0,98.497452,29.290000,0.00000,1.0,1.00000,0.070736,0.000000,1.0


In [22]:
X.shape,Y.shape

((1000, 42), (1000,))

In [12]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
ffs = f_regression(X,Y)
variable = [ ]
for j in range(1,20):
    a=[]
    for i in range(0,len(X.columns)-1):
        if ffs[0][i] >=j*10:
            a.append(X.columns[i])
    variable.append(a)
# >>> X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
# >>> lreg = LinearRegression()
# # >>> rfe = RFE(lreg, 10)
# >>> selector = RFE(lreg ,33, step=1)
# >>> selector = selector.fit(X, Y)
# print('score',selector.score(X,Y))
# # >>> print(selector.support_)
# >>> selector.ranking_

In [13]:
# for j in range(1,len(variable)+1):
k=[i*10 for i in range(1,20)]
len(variable)
#     print(j*10, variable[j])
df=pd.DataFrame(variable,index=k)

In [20]:
variable[10]

['mvar1', 'mvar42']

In [14]:
# i=1

new_X=X[variable[10]]
#     print(new_X.shape)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
   new_X ,Y , test_size=0.33, random_state=42)
X_train.shape, Y_train.shape, X_test.shape
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
knn_cost=cross_val_score(knn, X_train, Y_train, scoring='neg_log_loss').mean()
print(acc_knn,knn_cost)
# i+=1
# variable[0]

87.31 -3.2019963521857506


In [23]:
lead_df1=pd.read_csv('lead_board_cleaned.csv',index_col='application_key')

In [25]:
# lead_df=lead_df1[variable[10]]
colm=X.columns

(42,)

In [26]:
lead_df.head()

,mvar1,mvar42
application_key,,
350053,1753.0,0.42105
350054,1894.0,0.00000
350055,1774.0,0.33333
350056,1586.0,1.50000
350057,1832.0,0.00000


In [9]:
lead_df.shape

(25000, 44)

In [105]:
lead_pred=knn.predict(lead_df)
proba=knn.predict_proba(lead_df)
lead_df['pridicted']=lead_pred
lead_df['prob']=proba[:,0]
lead_df=lead_df.sort_values(by=['prob'],ascending=False)
result=lead_df['pridicted'].astype(int)
result.to_csv('Datadevils_IITGuwahati_9.csv')

C:\Users\BaadRoR\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\BaadRoR\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [106]:
lead_df.head(10)

,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,mvar13,mvar15,mvar16,mvar17,mvar18,mvar19,mvar20,mvar21,mvar22,mvar23,mvar25,mvar26,mvar27,mvar28,mvar29,mvar30,mvar31,mvar32,mvar33,mvar34,mvar35,mvar36,mvar37,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,pridicted,prob
application_key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
350053,1753.0,0.5001,0.000,0.0,0.000,206.0,19179.0,206.0,14221.0,19438.0,1.492284e+04,3.758300e+05,0.0,0.0,0.0,1.0,1.0,79.200,7920.00000,7920.000000,1141.0,152.0,152.0,1551.0,8486.0,4045.000000,3661.680565,450.0,13.0833,0.0,0.000000,3.0,11.0,1.0,0.0,98.49114,20.551000,0.42105,3.0,0.85661,0.07128,0.000000,0.0,1.0
363950,1762.0,0.6206,0.000,0.0,0.000,380.0,4955.0,4955.0,9389.0,12854.0,7.561000e+03,2.963090e+05,0.0,0.0,0.0,1.0,1.0,80.850,1667.01000,1667.010000,2201.0,274.0,274.0,913.0,6692.0,2598.012914,3661.680565,796.0,1.0833,0.0,0.000000,2.0,7.0,6.0,0.0,98.49114,71.990942,0.35000,4.0,0.49161,0.07128,0.000000,0.0,1.0
363891,1844.0,0.2662,0.000,0.0,0.000,4955.0,31712.0,31712.0,121100.0,30611.0,1.354900e+04,6.639700e+05,0.0,0.0,0.0,0.0,0.0,6.603,11.94000,0.000000,1489.0,1886.0,700.0,30386.0,8882.0,2598.012914,3661.680565,881.0,4.0000,0.0,0.000000,1.0,4.0,10.0,0.0,98.49114,71.990942,0.00000,6.0,0.95576,0.07128,0.092712,0.0,1.0
363892,1847.0,0.4201,0.000,0.0,0.000,297.0,16552.0,16552.0,182869.0,77003.0,1.465300e+04,1.189200e+06,0.0,0.0,0.0,0.0,0.0,43.567,0.00000,0.000000,2598.0,6844.0,6144.0,1308.0,6905.0,2598.012914,3661.680565,2049.0,4.8333,0.0,0.000000,2.0,11.0,8.0,0.0,98.49114,71.990942,0.02703,20.0,0.97446,0.00000,0.000000,0.0,1.0
363894,1747.0,3.7505,0.000,0.0,0.000,0.0,27444.0,27444.0,2023.0,5126.0,6.746700e+04,4.045190e+05,1.0,1.0,1.0,3.0,3.0,10.758,7362.96000,6822.730000,923.0,426.0,426.0,517.0,3011.0,2598.012914,3661.680565,91.0,0.1667,0.0,0.000000,6.0,0.0,9.0,0.0,98.49114,71.990942,0.20000,7.0,0.50028,0.07128,0.000000,0.0,1.0
363895,1726.0,2.7551,0.000,0.0,0.000,77.0,991.0,991.0,2081.0,1814.0,2.797000e+03,2.936359e+05,0.0,0.0,0.0,2.0,2.0,82.583,2771.43000,1000.000000,281.0,426.0,426.0,1582.0,1795.0,2598.012914,1795.000000,37.0,3.9167,0.0,0.000000,7.0,0.0,9.0,0.0,100.00000,71.990942,0.20000,8.0,0.57686,0.07128,0.000000,0.0,1.0
363899,1657.0,0.8968,2.679,0.0,0.000,0.0,5118.0,251.0,4806.0,15888.0,6.942000e+03,8.046700e+04,1.0,1.0,1.0,2.0,2.0,92.361,8735.00000,4270.918441,872.0,1278.0,1278.0,0.0,3346.0,913.000000,3661.680565,86.0,3.3333,2.0,0.000000,9.0,0.0,4.0,0.0,98.49114,91.287000,0.16667,9.0,0.38464,0.07128,0.000000,0.0,1.0
363900,1673.0,0.1056,16.011,0.0,5.489,491.0,19873.0,1103.0,15321.0,21466.0,1.492284e+04,2.078130e+05,0.0,0.0,0.0,0.0,0.0,42.571,4427.87677,4270.918441,4187.0,5810.0,5810.0,0.0,8091.0,2598.012914,3741.000000,1239.0,7.9167,1.0,0.200243,7.0,7.0,0.0,0.0,134.40100,71.990942,0.40909,3.0,0.61889,1.00000,0.092712,0.0,1.0
363901,1833.0,0.0272,0.233,0.0,0.000,1103.0,37042.0,9268.0,70575.0,106246.0,1.491175e+06,2.936359e+05,0.0,0.0,0.0,0.0,1.0,30.876,4205.19000,5677.000000,2572.0,6144.0,6144.0,243.0,9277.0,2598.012914,3661.680565,1522.0,20.7500,0.0,0.000000,10.0,12.0,4.0,0.0,98.49114,71.990942,0.03333,17.0,0.54948,0.07128,0.092712,0.0,1.0


In [109]:
new_X=pd.concat([new_X,Y],axis=1)

In [110]:
new_X.shape

(80000, 43)

In [112]:
new_X.to_csv('Train_after_forward.csv')